In [1]:
import numpy as np
import faiss
from functions import create_color_histogram, create_vit_embedding, get_glcm_features, extract_hog_features, extract_cnn_features
import pandas as pd
import cv2
import os

In [2]:
color_histograms_index = faiss.read_index("database/color_histograms.index") 
texture_histograms_index = faiss.read_index("database/histograms_texture.index")
hog_index = faiss.read_index("database/hog.index")
cnn_index = faiss.read_index("database/cnn.index")
embeddings_index = faiss.read_index("database/vit.index")

In [13]:
def precision_at_k(query, k, index, ground_truth):
    faiss.normalize_L2(query)
    _, I = index.search(query, k)
    return len(set(I[0]) & ground_truth) / k

In [4]:
train_db = pd.read_csv("database/db.csv", index_col=0)
test_db = pd.read_csv("database/db_test.csv", index_col=0)

In [5]:
ground_truth_dic = {"Coast": set(range(100)), 
                    "Desert": set(range(100, 200)), 
                    "Forest": set(range(200, 300)), 
                    "Glacier": set(range(300, 400)), 
                    "Mountain": set(range(400, 500))}

In [ ]:
def evaluate_precision_at_k(func, index, test_db, k):
    precisions = []
    for test_image, label in test_db[['image', 'label']].values:
        path = "images/" + test_image
        img = cv2.imread(path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        query = func(img_rgb).reshape(1, -1)
        ground_truth = ground_truth_dic[label]
        precisions.append(precision_at_k(query, k, index, ground_truth))
    return np.mean(precisions)

In [25]:
def mean_averaged_precision_at_k(func, index, test_db, k):
    precisions = []
    for test_image, label in test_db[['image', 'label']].values:
        path = "images/" + test_image
        img = cv2.imread(path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        query = func(img_rgb).reshape(1, -1)
        ground_truth = ground_truth_dic[label]
        precision = precision_at_k(query, k, index, ground_truth)
        precisions.append(precision)
    return np.mean(precisions)

In [7]:
# Evitar conflicto de OpenMP
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Desactiva GPU en TensorFlow

In [18]:
evaluate_precision_at_k(create_vit_embedding, embeddings_index, test_db, 3)

0.8466666666666666

In [28]:
methods = ["Color Histogram", "Texture Histogram", "HOG", "CNN", "VIT"]
funcs = [create_color_histogram, get_glcm_features, extract_hog_features, extract_cnn_features, create_vit_embedding]
indexes = [color_histograms_index, texture_histograms_index, hog_index, cnn_index, embeddings_index]
results = {}

for i in range(5):
    method = methods[i]
    func = funcs[i]
    index = indexes[i]
    precision_k = evaluate_precision_at_k(func, index, test_db, 10)
    mean_ap_k = mean_averaged_precision_at_k(func, index, test_db, 10)
    results[method] = [precision_k, mean_ap_k]

df_results = pd.DataFrame(results, index=["Precision@3", "Mean AP@3"])

1/1 [==============================] - 0s 130ms/step


In [29]:
df_results

,Color Histogram,Texture Histogram,HOG,CNN,VIT
Precision@3,0.37,0.338,0.26,0.198,0.842
Mean AP@3,0.37,0.338,0.26,0.198,0.842
